# Mutaciones 
El siguiente programa computa las semillas formuladas a partir del siguiente vector de polinomios $S_0 := {((1,x),(1+x^2, 1+x+x^2))}$ con todos los posibles vectores del campo $\mathbb{F}:= [\mathbb{Z_2}]/<x^3+x+1>$, compaando asi la vida de cada semilla generada.

In [23]:
from sympy import symbols, Poly, GF, div
import numpy as np

# Definir el símbolo y el campo finito Z_2
x = symbols('x')

# Definición del campo Z_2 y divisor
campo_finito = GF(2)
divisor = x ** 3 + x + 1


Hacemos uso de la libreria numpy para manipular matrices y sympy para la representacion de cada polinomio, con una ventaja especial GF en sympy que nos permite realizar operaciones modulo 2 haciendo el campo finito a $\mathbb{Z_2}$, definimos una unica variable $x$ y el divisor del campo $x^3+x+1$.<br>
<br>
Definimos las siguientes funciones que nos permitiran automatizar el proceso de mutar las palabras originales con los distintos vectores, consideramos una funcion que nos permita representar los polinomios en binario. T_w_sk_1 función que cumpla el papel de la transformación Affine $T(\omega(sk+1))$ donde $\omega_i$ corresponde a un poligono y se halla al realizar producto punto entre la matriz $A$ y el poligono dado. omega_prime nos permite calcular el siguiente poligono $\omega_{i+1}$ a partir de polinomios ya formados y por ultimo consideramos la función residuo que nos permite mantener el campo finito cerrado.


In [24]:
# Función para la representación binaria
def poly_a_binario(poly):
    # Obtener los coeficientes del polinomio
    coeficientes = poly.all_coeffs()

    # Convertir los coeficientes a binario y unirlos en una cadena
    representacion_binaria = ''.join([str(int(coeficiente)) for coeficiente in coeficientes])

    # Rellenar con ceros a la izquierda si es necesario
    representacion_binaria = representacion_binaria.zfill(3)

    return representacion_binaria


# Función para definir la funcion T(w(sk-1))
def T_w_sk_1(matrix_A, omega_1, vs):
    matrix_A_w1 = matrix_A.dot(omega_1)
    return matrix_A_w1 + vs


# Función para calcular w_0 + T(w(sk-1)) 
def omega_prime(omega_0, T_w):
    return omega_0 + T_w


# Reducción del polinomio dado el divisor 
def residuo(w, divisor):
    mutacion_n = []
    for i in w:
        for polinomio in i:
            cociente, res = div(polinomio, divisor, domain=campo_finito)
            mutacion_n.append([res])
    return np.array(mutacion_n)



Definimos los polinomios y sus respectivos poligonos originales en conjunto con la matriz $A$ a usar.

In [25]:
# Definición de los polinomios de la tarea
polinomio_1 = Poly(1, x, domain=campo_finito)
polinomio_2 = Poly(x, x, domain=campo_finito)

polinomio_3 = Poly(1 + x**2, x, domain=campo_finito)
polinomio_4 = Poly(1 + x + x ** 2, x, domain=campo_finito)

# Poligonos iniciales
w_0 = np.array([[polinomio_1], [polinomio_2]])
w_1 = np.array([[polinomio_3], [polinomio_4]])

# Mostramos los poligonos en pantalla
print(f'Elementos de la semilla original')
print(f'x_0 = ({polinomio_1.as_expr(), polinomio_2.as_expr()} {polinomio_3.as_expr(), polinomio_4.as_expr()})')
print(f'w(0) = {poly_a_binario(polinomio_1) + poly_a_binario(polinomio_2)}')
print(f'w(1) = {poly_a_binario(polinomio_3) + poly_a_binario(polinomio_4)}')

# Definición de la matrix A
polinomio_5 = Poly(1 + x, x, domain=campo_finito)
polinomio_6 = Poly(x + x ** 2, x, domain=campo_finito)
matrix_A = np.array([[polinomio_1, polinomio_5], [polinomio_6, polinomio_2]])
print('\nMatriz A definida')
for i in matrix_A:
    print(f'[{i[0].as_expr()},{i[1].as_expr()}]')

Elementos de la semilla original
x_0 = ((1, x) (x**2 + 1, x**2 + x + 1))
w(0) = 001010
w(1) = 101111

Matriz A definida
[1,x + 1]
[x**2 + x,x]


Primera mutación bajo el vector $vs = [0,0]$, nos da una vida de 64 mutaciones, a continuación de muestra el resultado de cada iteración.

In [26]:
# Definimos el vs a mirar
polinomio_7 = Poly(0, x, domain=campo_finito)
vs = np.array([[polinomio_7], [polinomio_7]])
print('\nVector VS definido')
for i in vs:
    print(i[0].as_expr())
print('\n')
i = 0
w_0_copy = w_0
w_1_copy = w_1
while i >= 0:
    T_w = T_w_sk_1(matrix_A=matrix_A, omega_1=w_1_copy, vs=vs)
    w_i = omega_prime(omega_0=w_0_copy, T_w=T_w)
    mutacion_i = residuo(w_i, divisor)

    w_0_copy = w_1_copy
    w_1_copy = mutacion_i

    print(f'Elementos de la mutación {i}')
    print(
        f'x_{i} = ({w_0_copy[0, 0].as_expr(), w_0_copy[1, 0].as_expr()}, {w_1_copy[0, 0].as_expr(), w_1_copy[1, 0].as_expr()})')
    print(f'w(0) = {poly_a_binario(w_0_copy[0, 0]) + poly_a_binario(w_0_copy[1, 0])}')
    print(f'w(1) = {poly_a_binario(w_1_copy[0, 0]) + poly_a_binario(w_1_copy[1, 0])}')
    i += 1
    if np.array_equal(w_0_copy, w_0) and np.array_equal(w_1_copy, w_1):
        break

print("\nVida de la semilla con la matriz A y el vector VS:",i)


Vector VS definido
0
0


Elementos de la mutación 0
x_0 = ((x**2 + 1, x**2 + x + 1), (x**2 + x, x**2))
w(0) = 101111
w(1) = 110100
Elementos de la mutación 1
x_1 = ((x**2 + x, x**2), (x**2, x**2 + x))
w(0) = 110100
w(1) = 100110
Elementos de la mutación 2
x_2 = ((x**2, x**2 + x), (x + 1, x**2 + x))
w(0) = 100110
w(1) = 011110
Elementos de la mutación 3
x_3 = ((x + 1, x**2 + x), (x**2 + x, 0))
w(0) = 011110
w(1) = 110000
Elementos de la mutación 4
x_4 = ((x**2 + x, 0), (x**2 + 1, x**2))
w(0) = 110000
w(1) = 101100
Elementos de la mutación 5
x_5 = ((x**2 + 1, x**2), (x**2, 0))
w(0) = 101100
w(1) = 100000
Elementos de la mutación 6
x_6 = ((x**2, 0), (1, 1))
w(0) = 100000
w(1) = 001001
Elementos de la mutación 7
x_7 = ((1, 1), (x**2 + x, x**2))
w(0) = 001001
w(1) = 110100
Elementos de la mutación 8
x_8 = ((x**2 + x, x**2), (0, 0))
w(0) = 110100
w(1) = 000000
Elementos de la mutación 9
x_9 = ((0, 0), (x**2 + x, x**2))
w(0) = 000000
w(1) = 110100
Elementos de la mutación 10
x_10 = ((x**2 + 

Primera mutación bajo el vector $vs = [1,1]$, nos da una vida de 64 mutaciones, a continuación de muestra el resultado de cada iteración.

In [27]:
# Definimos el vs a mirar
polinomio_7 = Poly(1, x, domain=campo_finito)
vs = np.array([[polinomio_7], [polinomio_7]])
print('\nVector VS definido')
for i in vs:
    print(i[0].as_expr())
print('\n')

i = 0
w_0_copy = w_0
w_1_copy = w_1
while i >= 0:
    T_w = T_w_sk_1(matrix_A=matrix_A, omega_1=w_1_copy, vs=vs)
    w_i = omega_prime(omega_0=w_0_copy, T_w=T_w)
    mutacion_i = residuo(w_i, divisor)

    w_0_copy = w_1_copy
    w_1_copy = mutacion_i

    print(f'Elementos de la mutación {i}')
    print(
        f'x_{i} = ({w_0_copy[0, 0].as_expr(), w_0_copy[1, 0].as_expr()}, {w_1_copy[0, 0].as_expr(), w_1_copy[1, 0].as_expr()})')
    print(f'w({i}) = {poly_a_binario(w_0_copy[0, 0]) + poly_a_binario(w_0_copy[1, 0])}')
    print(f'w({i+1}) = {poly_a_binario(w_1_copy[0, 0]) + poly_a_binario(w_1_copy[1, 0])}')
    i += 1
    if np.array_equal(w_0_copy, w_0) and np.array_equal(w_1_copy, w_1):
        break

print("\nVida de la semilla con la matriz A y el vector VS:",i)


Vector VS definido
1
1


Elementos de la mutación 0
x_0 = ((x**2 + 1, x**2 + x + 1), (x**2 + x + 1, x**2 + 1))
w(0) = 101111
w(1) = 111101
Elementos de la mutación 1
x_1 = ((x**2 + x + 1, x**2 + 1), (x**2 + x + 1, x + 1))
w(1) = 111101
w(2) = 111011
Elementos de la mutación 2
x_2 = ((x**2 + x + 1, x + 1), (x**2, x**2 + x))
w(2) = 111011
w(3) = 100110
Elementos de la mutación 3
x_3 = ((x**2, x**2 + x), (x + 1, 0))
w(3) = 100110
w(4) = 011000
Elementos de la mutación 4
x_4 = ((x + 1, 0), (x**2 + x, x**2 + x))
w(4) = 011000
w(5) = 110110
Elementos de la mutación 5
x_5 = ((x**2 + x, x**2 + x), (x**2 + 1, x**2))
w(5) = 110110
w(6) = 101100
Elementos de la mutación 6
x_6 = ((x**2 + 1, x**2), (x**2 + 1, x**2 + x + 1))
w(6) = 101100
w(7) = 101111
Elementos de la mutación 7
x_7 = ((x**2 + 1, x**2 + x + 1), (x + 1, x + 1))
w(7) = 101111
w(8) = 011011
Elementos de la mutación 8
x_8 = ((x + 1, x + 1), (x, 1))
w(8) = 011011
w(9) = 010001
Elementos de la mutación 9
x_9 = ((x, 1), (x + 1, x**2 + x +

x_18 = ((x**2 + x + 1, x**2 + x), (0, x + 1))
w(18) = 111110
w(19) = 000011
Elementos de la mutación 19
x_19 = ((0, x + 1), (x + 1, 1))
w(19) = 000011
w(20) = 011001
Elementos de la mutación 20
x_20 = ((x + 1, 1), (1, 1))
w(20) = 011001
w(21) = 001001
Elementos de la mutación 21
x_21 = ((1, 1), (0, x**2))
w(21) = 001001
w(22) = 000100
Elementos de la mutación 22
x_22 = ((0, x**2), (x**2 + x + 1, x + 1))
w(22) = 000100
w(23) = 111011
Elementos de la mutación 23
x_23 = ((x**2 + x + 1, x + 1), (x + 1, x**2 + x + 1))
w(23) = 111011
w(24) = 011111
Elementos de la mutación 24
x_24 = ((x + 1, x**2 + x + 1), (x**2 + x + 1, x**2 + x))
w(24) = 011111
w(25) = 111110
Elementos de la mutación 25
x_25 = ((x**2 + x + 1, x**2 + x), (x**2, x**2 + 1))
w(25) = 111110
w(26) = 100101
Elementos de la mutación 26
x_26 = ((x**2, x**2 + 1), (x**2 + x, x + 1))
w(26) = 100101
w(27) = 110011
Elementos de la mutación 27
x_27 = ((x**2 + x, x + 1), (x**2 + x, 0))
w(27) = 110011
w(28) = 110000
Elementos de la mutació

Primera mutación bajo el vector $vs = [0,1]$, nos da una vida de 64 mutaciones, a continuación de muestra el resultado de cada iteración.

In [28]:
# Definimos el vs a mirar
polinomio_7 = Poly(0, x, domain=campo_finito)
polinomio_8 = Poly(1, x, domain=campo_finito)
vs = np.array([[polinomio_7], [polinomio_8]])
print('\nVector VS definido')
for i in vs:
    print(i[0].as_expr())
print('\n')

i = 0
w_0_copy = w_0
w_1_copy = w_1
while i >= 0:
    T_w = T_w_sk_1(matrix_A=matrix_A, omega_1=w_1_copy, vs=vs)
    w_i = omega_prime(omega_0=w_0_copy, T_w=T_w)
    mutacion_i = residuo(w_i, divisor)

    w_0_copy = w_1_copy
    w_1_copy = mutacion_i

    print(f'Elementos de la mutación {i}')
    print(
        f'x_{i} = ({w_0_copy[0, 0].as_expr(), w_0_copy[1, 0].as_expr()}, {w_1_copy[0, 0].as_expr(), w_1_copy[1, 0].as_expr()})')
    print(f'w({i}) = {poly_a_binario(w_0_copy[0, 0]) + poly_a_binario(w_0_copy[1, 0])}')
    print(f'w({i+1}) = {poly_a_binario(w_1_copy[0, 0]) + poly_a_binario(w_1_copy[1, 0])}')
    i += 1
    if np.array_equal(w_0_copy, w_0) and np.array_equal(w_1_copy, w_1):
        break

print("\nVida de la semilla con la matriz A y el vector VS:",i)


Vector VS definido
0
1


Elementos de la mutación 0
x_0 = ((x**2 + 1, x**2 + x + 1), (x**2 + x, x**2 + 1))
w(0) = 101111
w(1) = 110101
Elementos de la mutación 1
x_1 = ((x**2 + x, x**2 + 1), (x**2 + x + 1, x**2 + 1))
w(1) = 110101
w(2) = 111101
Elementos de la mutación 2
x_2 = ((x**2 + x + 1, x**2 + 1), (x**2 + 1, 1))
w(2) = 111101
w(3) = 101001
Elementos de la mutación 3
x_3 = ((x**2 + 1, 1), (1, x**2 + 1))
w(3) = 101001
w(4) = 001101
Elementos de la mutación 4
x_4 = ((1, x**2 + 1), (0, x**2 + x + 1))
w(4) = 001101
w(5) = 000111
Elementos de la mutación 5
x_5 = ((0, x**2 + x + 1), (x + 1, 1))
w(5) = 000111
w(6) = 011001
Elementos de la mutación 6
x_6 = ((x + 1, 1), (0, x**2 + 1))
w(6) = 011001
w(7) = 000101
Elementos de la mutación 7
x_7 = ((0, x**2 + 1), (x**2 + x + 1, 1))
w(7) = 000101
w(8) = 111001
Elementos de la mutación 8
x_8 = ((x**2 + x + 1, 1), (x**2, x))
w(8) = 111001
w(9) = 100010
Elementos de la mutación 9
x_9 = ((x**2, x), (x**2 + 1, 1))
w(9) = 100010
w(10) = 101001
Elem

Primera mutación bajo el vector $vs = [1,0]$, nos da una vida de 64 mutaciones, a continuación de muestra el resultado de cada iteración.

In [29]:
# Definimos el vs a mirar
polinomio_7 = Poly(0, x, domain=campo_finito)
polinomio_8 = Poly(1, x, domain=campo_finito)
vs = np.array([[polinomio_8], [polinomio_7]])
print('\nVector VS definido')
for i in vs:
    print(i[0].as_expr())
print('\n')

i = 0
w_0_copy = w_0
w_1_copy = w_1
while i >= 0:
    T_w = T_w_sk_1(matrix_A=matrix_A, omega_1=w_1_copy, vs=vs)
    w_i = omega_prime(omega_0=w_0_copy, T_w=T_w)
    mutacion_i = residuo(w_i, divisor)

    w_0_copy = w_1_copy
    w_1_copy = mutacion_i

    print(f'Elementos de la mutación {i}')
    print(
        f'x_{i} = ({w_0_copy[0, 0].as_expr(), w_0_copy[1, 0].as_expr()}, {w_1_copy[0, 0].as_expr(), w_1_copy[1, 0].as_expr()})')
    print(f'w({i}) = {poly_a_binario(w_0_copy[0, 0]) + poly_a_binario(w_0_copy[1, 0])}')
    print(f'w({i+1}) = {poly_a_binario(w_1_copy[0, 0]) + poly_a_binario(w_1_copy[1, 0])}')
    i += 1
    if np.array_equal(w_0_copy, w_0) and np.array_equal(w_1_copy, w_1):
        break

print("\nVida de la semilla con la matriz A y el vector VS:",i)


Vector VS definido
1
0


Elementos de la mutación 0
x_0 = ((x**2 + 1, x**2 + x + 1), (x**2 + x + 1, x**2))
w(0) = 101111
w(1) = 111100
Elementos de la mutación 1
x_1 = ((x**2 + x + 1, x**2), (x**2, 0))
w(1) = 111100
w(2) = 100000
Elementos de la mutación 2
x_2 = ((x**2, 0), (x, 1))
w(2) = 100000
w(3) = 010001
Elementos de la mutación 3
x_3 = ((x, 1), (x**2, x**2 + 1))
w(3) = 010001
w(4) = 100101
Elementos de la mutación 4
x_4 = ((x**2, x**2 + 1), (x + 1, x**2 + 1))
w(4) = 100101
w(5) = 011101
Elementos de la mutación 5
x_5 = ((x + 1, x**2 + 1), (x, x**2 + 1))
w(5) = 011101
w(6) = 010101
Elementos de la mutación 6
x_6 = ((x, x**2 + 1), (x**2, x + 1))
w(6) = 010101
w(7) = 100011
Elementos de la mutación 7
x_7 = ((x**2, x + 1), (x, x**2 + x))
w(7) = 100011
w(8) = 010110
Elementos de la mutación 8
x_8 = ((x, x**2 + x), (x**2 + x, x + 1))
w(8) = 010110
w(9) = 110011
Elementos de la mutación 9
x_9 = ((x**2 + x, x + 1), (0, x))
w(9) = 110011
w(10) = 000010
Elementos de la mutación 10
x_10 = 